In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('project').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('/home/aravind/Downloads/sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr =LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

In [10]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.meanAbsoluteError

8.145215527783876

In [14]:
training_summary.r2

0.027839179518600154

In [15]:
training_summary.rootMeanSquaredError

10.16309157133015

In [16]:
all_data = spark.read.format('libsvm').load('/home/aravind/Downloads/sample_linear_regression_data.txt')

In [17]:
train_data, Test_data = all_data.randomSplit([0.8,0.2])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                405|
|   mean| 0.5033229364966743|
| stddev| 10.389265062627965|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
Test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                 96|
|   mean|-0.7827550975152414|
| stddev|   9.99711991952011|
|    min|-26.805483428483072|
|    max| 19.174935630244647|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

In [24]:
test_results = correct_model.evaluate(Test_data)

In [33]:
test_results.rootMeanSquaredError

10.156944917564587

In [26]:
unlabeled_data = Test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = correct_model.transform(unlabeled_data)

In [31]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|  2.428879717050768|
|(10,[0,1,2,3,4,5,...| 1.0127096690322035|
|(10,[0,1,2,3,4,5,...|-2.1073535794424267|
|(10,[0,1,2,3,4,5,...|-0.7346679297691456|
|(10,[0,1,2,3,4,5,...| 1.6142329524884267|
|(10,[0,1,2,3,4,5,...|-1.6075802151883014|
|(10,[0,1,2,3,4,5,...| 2.3165982506409915|
|(10,[0,1,2,3,4,5,...| 0.9005414051591913|
|(10,[0,1,2,3,4,5,...|-2.2395101457827065|
|(10,[0,1,2,3,4,5,...|  2.791599453829139|
|(10,[0,1,2,3,4,5,...|   1.51407667313863|
|(10,[0,1,2,3,4,5,...|  2.282398673574434|
|(10,[0,1,2,3,4,5,...| 1.5920219902931043|
|(10,[0,1,2,3,4,5,...|-3.1734816529343384|
|(10,[0,1,2,3,4,5,...| 1.8167561814530995|
|(10,[0,1,2,3,4,5,...| 0.6861303429345658|
|(10,[0,1,2,3,4,5,...|-1.2561583753092533|
|(10,[0,1,2,3,4,5,...| -2.208902679835149|
|(10,[0,1,2,3,4,5,...| 0.6144889644639505|
|(10,[0,1,2,3,4,5,...| -2.550247724028526|
+----------